# Investigate Coffee Shop Sales

This sample dataset contains sales and transaction data from January through June. I downloaded the dataset from Kaggle, and utilized SQL to answer the following questions about the dataset:

1. Do sales change by month?
2. What product categories are people buying? Which are the most profitable?
3. What product categories are being bought at what times throughout the day?
4. Do sales change by the day of the week?
5. How is each of the three company store locations performing?
6. What are the 10 best-selling products?
7. Rank the top selling products in each of the product categories.

Let's see what we can find out.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sales = pd.read_excel('coffee_shop_original.xlsx')

In [2]:
SELECT *
FROM sales

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail
0,1,2023-01-01 00:00:00+00:00,07:06:11,2,5,Lower Manhattan,32,3.00,Coffee,Gourmet brewed coffee,Ethiopia Rg
1,2,2023-01-01 00:00:00+00:00,07:08:56,2,5,Lower Manhattan,57,3.10,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg
2,3,2023-01-01 00:00:00+00:00,07:14:04,2,5,Lower Manhattan,59,4.50,Drinking Chocolate,Hot chocolate,Dark chocolate Lg
3,4,2023-01-01 00:00:00+00:00,07:20:24,1,5,Lower Manhattan,22,2.00,Coffee,Drip coffee,Our Old Time Diner Blend Sm
4,5,2023-01-01 00:00:00+00:00,07:22:41,2,5,Lower Manhattan,57,3.10,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg
...,...,...,...,...,...,...,...,...,...,...,...
149111,149452,2023-06-30 00:00:00+00:00,20:18:41,2,8,Hell's Kitchen,44,2.50,Tea,Brewed herbal tea,Peppermint Rg
149112,149453,2023-06-30 00:00:00+00:00,20:25:10,2,8,Hell's Kitchen,49,3.00,Tea,Brewed Black tea,English Breakfast Lg
149113,149454,2023-06-30 00:00:00+00:00,20:31:34,1,8,Hell's Kitchen,45,3.00,Tea,Brewed herbal tea,Peppermint Lg
149114,149455,2023-06-30 00:00:00+00:00,20:57:19,1,8,Hell's Kitchen,40,3.75,Coffee,Barista Espresso,Cappuccino


## 1. Do sales change by month?

The query below groups the number of transactions per month, as well as the total dollar sales per month. 

In [3]:
SELECT
	EXTRACT(month FROM transaction_date) AS month,
	COUNT(transaction_id) AS transactions,
	ROUND(SUM(transaction_qty * unit_price),0) AS sales
FROM sales
GROUP BY month
ORDER BY month

,month,transactions,sales
0,1,17314,81678.0
1,2,16359,76145.0
2,3,21229,98835.0
3,4,25335,118941.0
4,5,33527,156728.0
5,6,35352,166486.0


## 2. What product categories are people buying, and which are the most profitable?

The query below groups the number of transactions per product category, as well as total dollars sales by product category, then calculates dollars revenue per transaction.

In [4]:
SELECT
	product_category,
	COUNT(transaction_id) AS transactions,
	ROUND(SUM(transaction_qty * unit_price),0) AS sales,
	ROUND(SUM(transaction_qty * unit_price) / COUNT(transaction_id), 2) AS dollars_per_transaction
FROM sales
GROUP BY product_category
ORDER BY sales DESC

,product_category,transactions,sales,dollars_per_transaction
0,Coffee,58416,269952.0,4.62
1,Tea,45449,196406.0,4.32
2,Bakery,22796,82316.0,3.61
3,Drinking Chocolate,11468,72416.0,6.31
4,Coffee beans,1753,40085.0,22.87
5,Branded,747,13607.0,18.22
6,Loose Tea,1210,11214.0,9.27
7,Flavours,6790,8409.0,1.24
8,Packaged Chocolate,487,4408.0,9.05


## 3. What product categories are people buying and at what times are they buying them?

The query below is similar to the above query, however it takes that one step further. This query identifies total dollar sales per product category, per hour of the business day.

In [5]:
SELECT
	EXTRACT(hour FROM CAST(transaction_time AS TIME)) AS hour,
	product_category,
	COUNT(transaction_id) AS transactions
FROM sales
GROUP BY hour, product_category
ORDER BY transactions DESC

,hour,product_category,transactions
0,10,Coffee,7006
1,8,Coffee,6687
2,9,Coffee,6652
3,9,Tea,5296
4,10,Tea,5271
...,...,...,...
130,18,Packaged Chocolate,10
131,20,Coffee beans,9
132,20,Packaged Chocolate,7
133,20,Branded,4


## 4. What days are people buying?

The query below groups transaction counts by the day of week, and also sum of dollar sales by day of week.

In [6]:
SELECT
	EXTRACT(DOW FROM transaction_date) AS day_of_week,
	COUNT(transaction_id) AS count,
	ROUND(SUM(transaction_qty * unit_price),0) AS sales
FROM sales
GROUP BY EXTRACT(DOW FROM transaction_date)
ORDER BY day_of_week

,day_of_week,count,sales
0,0,21096,98330.0
1,1,21643,101677.0
2,2,21202,99456.0
3,3,21310,100314.0
4,4,21654,100768.0
5,5,21701,101373.0
6,6,20510,96894.0


## 5. How is each location performing?

The query below groups transaction count by store location, as well as total dollar sales by store location.

In [10]:
SELECT
	store_location,
	COUNT(transaction_id) AS transactions,
	ROUND(SUM(transaction_qty * unit_price),0) AS sales
FROM sales
GROUP BY store_location
ORDER BY sales DESC

,store_location,transactions,sales
0,Hell's Kitchen,50735,236511.0
1,Astoria,50599,232244.0
2,Lower Manhattan,47782,230057.0


## 6. What are the top 10 best-selling products?

The query below provides a count of the top 10 selling products, in descending order. There is also a 'rank' column, produced by a window function.

In [8]:
SELECT
	product_type,
	COUNT(product_type) AS count,
	RANK() OVER (ORDER BY count DESC) AS rank
FROM sales
GROUP BY product_type
LIMIT 10

,product_type,count,rank
0,Brewed Chai tea,17183,1
1,Gourmet brewed coffee,16912,2
2,Barista Espresso,16403,3
3,Hot chocolate,11468,4
4,Brewed Black tea,11350,5
5,Brewed herbal tea,11245,6
6,Scone,10173,7
7,Organic brewed coffee,8489,8
8,Drip coffee,8477,9
9,Premium brewed coffee,8135,10


## 7. What are the top-selling product types per product category?

The query below partitions the product categories, then provides the rank-ordered, top-selling product types per category.

In [12]:
SELECT
	product_category,
	product_type,
	COUNT(product_type) AS type_count,
	RANK() OVER(PARTITION BY product_category ORDER BY COUNT(product_type) DESC) AS rank
FROM sales
GROUP BY product_category, product_type

,product_category,product_type,type_count,rank
0,Coffee beans,Organic Beans,415,1
1,Coffee beans,Gourmet Beans,366,2
2,Coffee beans,Premium Beans,336,3
3,Coffee beans,Espresso Beans,319,4
4,Coffee beans,House blend Beans,183,5
5,Coffee beans,Green beans,134,6
6,Packaged Chocolate,Drinking Chocolate,266,1
7,Packaged Chocolate,Organic Chocolate,221,2
8,Coffee,Gourmet brewed coffee,16912,1
9,Coffee,Barista Espresso,16403,2


# Conclusions

1. It looks like people like to buy when its warm. June and May produced quite a bit more transactions and revenue than the other months. February and January produced the least. A full year's worth of data would provide some more insight into this question.

2. Coffee drinks produce the most transactions and most sales for the coffee shop, which makes sense. Tea drinks come in second not far behind. Bakery products are in third place, with about half the transactions and sales as tea. Drinking chocolate and coffee beans round out the top five. What's interesting is that coffee beans in the company's most profitable product but they don't sell too much of them. It could be beneficial for the company to improve marketing of the coffee beans.

3. A visualization would help with the results of this query, but we can see that the most transactions happen between 7am-11am and people are buying coffee and tea. Hopefully the coffee shop allocates the proper staffing levels during those busy times in the morning. Additionally, people seem to purchase coffee at high levels throughout the afternoon as well. Bakery products have their highest transactions between 7am-11am as well, however not nearly the high numbers as coffee and tea. Perhaps the coffee shop could market some combo meal specials to boost bakery sales.

4. Monday and Friday show the highest transactions and sales during the week. What surprised me is that Saturday and Sunday show the lowest transactions and sales. So it seems as though people are buying their coffees to help them get through the work day. 

5. Hell's Kitchen and Astoria are very close to each other in transactions and sales. Hell's Kitchen has just a small lead in both. Lower Manhattan seems to be trailing somewhat in the number of transactions, but not trailing so much in revenue. So Lower Manhattan must be doing a good job of selling the high margin products like coffee beans. In an effort to assist all three stores, perhaps the company could create a marketing campaign where a customer can get a discount on a hot coffee if they buy a bag of coffee beans, or vice versa.

6. Brewed chai tea is the top selling product. Who knew? Not me. Gourmet brewed coffee is second, followed by Barista espresso. The first non-drink item is scone and it's in overall seventh place. This result is another indicator that the coffee shop may do well to market some type of drink and food combo pricing deal.

7. It looks as though branded products and loose tea products don't achieve many sales transactions in this dataset, certainly not compared to the other categories. The coffee shop may want to re-evaluate these categories to see if there is a way to improve sales considerably, or maybe just drop the products altogeher.

## Resources

https://www.kaggle.com/datasets/ahmedabbas757/coffee-sales

https://medium.com/@hhuseyincosgun/learn-sql-window-functions-with-queries-3afb8118aa27

https://towardsdatascience.com/window-functions-a-must-know-for-data-engineers-and-data-scientists-4dd3e4ad0d2